# Классификация комментариев

In [1]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from pymystem3 import Mystem
from sklearn.metrics import f1_score

In [2]:
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Подготовка данных

In [3]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

toxic_comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Убедимся, что в данных нет пропусков:

In [4]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Посчитаем процент негативных комментариев в данных:

In [5]:
print(f'Количество негативных комментариев {toxic_comments["toxic"].value_counts().loc[1] / len(toxic_comments) * 100:.2f} %')

Количество негативных комментариев 10.17 %


Удалим из данных знаки препинания:

In [6]:
def clear_text(text):
    clear_list = re.sub(r'[^a-zA-z ]', ' ', text).split()
    clear_text = " ".join(clear_list)
    
    return clear_text

In [7]:
toxic_comments['stem_text'] = toxic_comments['text'].apply(clear_text)

Выделим леммы из строк:

In [8]:
snowball = SnowballStemmer(language='english')

def stemmer(text):
    token_text = word_tokenize(text)
    stem_text = " ".join([snowball.stem(word) for word in token_text])
    
    return stem_text

In [9]:
toxic_comments['stem_text'] = toxic_comments['stem_text'].progress_apply(stemmer)

Выделим признаки и целевой признак. Разделим данные на обучающую и тестовые выборки:

In [10]:
features = toxic_comments['stem_text']
target = toxic_comments['toxic']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=12345)

Рассчитаем величину TF-IDF для выборок:

In [11]:
train_corpus = X_train.values.astype('U')

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
X_train_tf_idf = count_tf_idf.fit_transform(train_corpus)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
test_corpus = X_test.values.astype('U')

X_test_tf_idf = count_tf_idf.transform(test_corpus)

Посчитаем процент негативных комментариев в выборках:

In [13]:
print(
    f'Количество негативных комментариев:\n\
    в обучающей выборке {y_train.value_counts().loc[1] / len(y_train) * 100:.2f} %\n\
    в тестовой выборке {y_test.value_counts().loc[1] / len(y_test) * 100:.2f} %'
)

Количество негативных комментариев:
    в обучающей выборке 10.16 %
    в тестовой выборке 10.19 %


### Вывод
В процессе подготовки из данных были выделены леммы, удалены знаки препинания и стоп-слова; рассчитаны величины TF-IDF; получены обучающая и тестовая выборки.

## Обучение

###  Логистическая регрессия

In [14]:
%%time
log_reg = LogisticRegression(random_state=12345, solver='liblinear')

cv_result = cross_validate(log_reg, X_train_tf_idf, y_train, cv=5, scoring='f1')

print(f'F1-score для модели логистической регрессии равно {cv_result["test_score"].mean():.2f}')

F1-score для модели логистической регрессии равно 0.72
CPU times: user 22.7 s, sys: 19.5 s, total: 42.1 s
Wall time: 42.2 s


In [15]:
log_reg.fit(X_train_tf_idf, y_train)
y_pred = log_reg.predict(X_test_tf_idf)

print(f'F1-score для модели логистической регрессии на тестовой выборке равно {f1_score(y_test, y_pred):.2f}')

F1-score для модели логистической регрессии на тестовой выборке равно 0.75


### Cтохастический градиентный спуск

In [16]:
%%time
sgd = SGDClassifier(random_state=12345)

param_grid = {
    "alpha": np.logspace(-5, 0, num=10),
    "penalty": ['l1', 'l2', 'elasticnet'],
}

grid_search = GridSearchCV(
    sgd,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
)

grid_search.fit(X_train_tf_idf, y_train)

print(f'F1-score модели градиентного спуска с параметрами {grid_search.best_params_} равно {(grid_search.best_score_):.2f}')

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


F1-score модели градиентного спуска с параметрами {'alpha': 1e-05, 'penalty': 'l1'} равно 0.78
CPU times: user 1min 4s, sys: 13.1 s, total: 1min 17s
Wall time: 1min 18s


In [17]:
y_pred = grid_search.predict(X_test_tf_idf)

print(f' F1-score модели градиентного спуска на тестовой выборке равно {f1_score(y_test, y_pred):.2f}')

 F1-score модели градиентного спуска на тестовой выборке равно 0.78


## Выводы

В результате удалось получить значение F1-score больше 0.75 на модели стохастического градиентного спуска.